<a href="https://colab.research.google.com/github/DIYDataScienceYT/KaggleCodes/blob/main/1_WritingQualityBaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Get Kaggle Dataset

In [1]:
! pip install kaggle --quiet

In [2]:
competition_name = "linking-writing-processes-to-writing-quality"

# https://www.kaggle.com/discussions/general/74235

# Mount your Google Drive.
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [6]:
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/Kaggle/kaggle_json/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [10]:
! kaggle competitions download -c {competition_name}

! mkdir kaggle_data
! unzip {competition_name + ".zip"} -d kaggle_data

# Unmount your Google Drive
drive.flush_and_unmount()


 95% 103M/108M [00:00<00:00, 191MB/s] 
100% 108M/108M [00:00<00:00, 178MB/s]
Archive:  linking-writing-processes-to-writing-quality.zip
  inflating: kaggle_data/sample_submission.csv  
  inflating: kaggle_data/test_logs.csv  
  inflating: kaggle_data/train_logs.csv  
  inflating: kaggle_data/train_scores.csv  


## Download requirements

In [ ]:
!pip install colorama catboost optuna -q

## Libraries

In [ ]:
%%time
# General library imports:-
# displaying styling elements (HTML, Markdown..)
from IPython.display import display_html, clear_output, Markdown;
# trigger manual garbage collection cycle
from gc import collect;
# working with dataframes
import pandas as pd;
# numerical and mathematical operations
import numpy as np;
# makes copy
from copy import deepcopy;
# frequence counter
from collections import Counter;
# regex - string operations
import re;

# process and os related imports:-
# mechanism for parallelizing the execution of functions
import joblib;
# os properties - run system commands (ls, cd), get process id, generate file hierarchy tree
from os import system, getpid, walk;
# interface for retrieving information on running processes and system utilization.
from psutil import Process;
# C library on Unix-like systems
import ctypes;
libc = ctypes.CDLL("libc.so.6");

# styling
# pretty-print: used to print asthetically
from pprint import pprint;
# Fore: color, Style: italics/bold
from colorama import Fore, Style, init;
# suppresses the display of warnings
from warnings import filterwarnings;
filterwarnings('ignore');

# visualizations
# extensible progress bar
from tqdm.autonotebook import tqdm;
# data visualization
import seaborn as sns;
import matplotlib.pyplot as plt;
# colormap from a list of colors
from matplotlib.colors import ListedColormap as LCM;
# command to print all plots within notebook, not new window
%matplotlib inline

print();
collect();


CPU times: user 647 ms, sys: 169 ms, total: 816 ms
Wall time: 1.04 s


4

In [ ]:
%%time

# cross-validation
from sklearn.model_selection import (RepeatedStratifiedKFold as RSKF,
                                     StratifiedKFold as SKF,
                                     KFold,
                                     RepeatedKFold as RKF,
                                     cross_val_score);
# ML Models
# logging evaluation results and early stopping during training
from lightgbm import log_evaluation, early_stopping, LGBMRegressor as LGBMR;
from xgboost import XGBRegressor as XGBR;
from catboost import CatBoostRegressor as CBR;
from sklearn.ensemble import (HistGradientBoostingRegressor as HGBR,
                              RandomForestRegressor as RFR,
                              ExtraTreesRegressor as ETR,
                              BaggingRegressor as BR
                             );
from sklearn.neighbors import KNeighborsRegressor as KNNR;
from sklearn.metrics import mean_squared_error as mse, make_scorer;

from sklearn.pipeline import Pipeline, make_pipeline;
from sklearn.impute import SimpleImputer;
# applying transformers to columns
from sklearn.compose import ColumnTransformer;
# FT: apply a custom function
from sklearn.preprocessing import (FunctionTransformer as FT,
                                   StandardScaler, RobustScaler, MinMaxScaler,MaxAbsScaler,
                                  );

import optuna;
from optuna import Trial, trial, create_study;
from optuna.samplers import TPESampler, CmaEsSampler;
optuna.logging.set_verbosity = optuna.logging.ERROR;

print();
collect();


CPU times: user 1.66 s, sys: 187 ms, total: 1.85 s
Wall time: 2.65 s


0

In [ ]:
%%time

# Defining global configurations and functions:-

# Setting rc parameters in seaborn for plots and graphs-
# Reference - https://matplotlib.org/stable/tutorials/introductory/customizing.html:-
# To alter this, refer to matplotlib.rcParams.keys()

sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.75,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : '#0099e6',
         'axes.titlesize'       : 8.5,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

# Color printing
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string";
    print(style + color + text + Style.RESET_ALL);

def GetMemUsage():
    """
    This function defines the memory usage across the kernel.
    Source-
    https://stackoverflow.com/questions/61366458/how-to-find-memory-usage-of-kaggle-notebook
    """;

    pid = getpid();
    py = Process(pid);
    memory_use = py.memory_info()[0] / 2. ** 30;
    return f"RAM memory GB usage = {memory_use :.4}";

# Making sklearn pipeline outputs as dataframe:-
from sklearn import set_config;
set_config(transform_output = "pandas");
pd.set_option('display.max_columns', 50);
pd.set_option('display.max_rows', 50);

print();
collect();


CPU times: user 177 ms, sys: 0 ns, total: 177 ms
Wall time: 182 ms


0

## Config

In [ ]:
%%time

# Configuration class:-
class CFG:
    """
    Configuration class for parameters and CV strategy for tuning and training
    Please use caps lock capital letters while filling in parameters
    """;

    # Data preparation:-
    version_nb         = 1;
    test_req           = "N";
    test_frac          = 0.01;
    load_tr_data       = "Y";
    gpu_switch         = "OFF";
    state              = 42;
    target             = 'score';

    path               = f"/kaggle/input/linking-writing-processes-to-writing-quality/";
    mydf_path          = f"/kaggle/input/writingqualitymemoryreduction/";

    # Model Training:-
    methods            = ["LGBM1R", "LGBM2R","XGBR", "CBR"];
    ML                 = "Y";
    scl_mthd           = "Robust";
    n_splits           = 5;
    n_repeats          = 5;
    nbrnd_erly_stp     = 25;
    mdlcv_mthd         = 'RSKF';

    # Ensemble:-
    ensemble_req       = "Y";
    enscv_mthd         = "RKF";
    metric_obj         = 'minimize';
    ntrials            = 10 if test_req == "Y" else 500;
    pstprcs_train      = "N";
    pstprcs_sub        = "N";

    # Global variables for plotting:-
    grid_specs     = {'visible': True, 'which': 'both', 'linestyle': '--',
                      'color': 'lightgrey', 'linewidth': 0.75
                     };
    title_specs    = {'fontsize': 9, 'fontweight': 'bold', 'color': 'tab:blue'};
    suptitle_specs = {'fontsize': 13, 'fontweight': 'bold', 'color': 'blue'};

print();
PrintColor(f"--> Configuration done!\n");
collect();

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);


--> Configuration done!


RAM memory GB usage = 0.2844
CPU times: user 172 ms, sys: 1.4 ms, total: 174 ms
Wall time: 249 ms


In [ ]:
%%time

# Commonly used CV strategies for later usage:-
all_cv= {'KF'  : KFold(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state),
         'RKF' : RKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'RSKF': RSKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'SKF' : SKF(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state)
        };

all_scaler = \
{"Z": StandardScaler(), "Robust": RobustScaler(),"MinMax": MinMaxScaler(),"MaxAbs": MaxAbsScaler()};

# Defining the competition metric:-
def ScoreMetric(ytrue, ypred)-> float:
    """
    This function calculates the metric for the competition.
    ytrue- ground truth array
    ypred- predictions
    returns - metric value (float)
    """;

    return mse(ytrue, ypred, squared = False);

# Designing a custom scorer to use in cross_val_predict and cross_val_score:-
myscorer = make_scorer(ScoreMetric, greater_is_better = False, needs_proba= False,);

# Making a post-process function for predictions:-
def PostProcessPred(pred, post_process: str):
    """
    This function post-processes the predictions as per user's discretion
    """;

    if post_process == "Y":
        return np.around(pred * 2) / 2 ;
    else:
        return pred;

print();
collect();

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);



RAM memory GB usage = 0.2844
CPU times: user 188 ms, sys: 2.04 ms, total: 190 ms
Wall time: 501 ms


## PreProcessors

In [ ]:
%%time

class Preprocessor():
    """
    Thanks to Marcus Chan for the class and the features
    Source- https://www.kaggle.com/code/mcpenguin/writing-processes-to-quality-baseline
    """;

    def __init__(self):
        self.seed = CFG.state;

        self.activities =\
        ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste'];
        self.events = \
        ['q', 'Space', 'Backspace', 'Shift',
         'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',',
         'ArrowDown', 'ArrowUp', 'Enter',
         'CapsLock', "'", 'Delete', 'Unidentified'
        ];
        self.text_changes = \
        ['q', ' ', 'NoChange', '.', ',', '\n', "'",
         '"', '-', '?', ';', '=', '/', '\\', ':'
        ];
        self.punctuations =\
        ['"', '.', ',', "'", '-', ';', ':',
         '?', '!', '<', '>', '/',
         '@', '#', '$', '%', '^',
         '&', '*', '(', ')', '_', '+'
        ];
        self.gaps = [1, 2, 3, 4, 5];

    def activity_counts(self, df):
        tmp_df = \
        df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols
        return ret

    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols
        return ret

    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols
        return ret

    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret


    def get_input_words(self, df):
        tmp_df = \
        df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = \
        tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = \
        tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = \
        tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = \
        tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = \
        tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = \
        tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = \
        tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df;

    def make_feats(self, df):

        PrintColor("\n---> Starting to engineer features <---\n", color = Fore.RED);

        # initialize features dataframe
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})

        # get shifted features
        # time shift
        PrintColor("\nEngineering time data\n")
        for gap in self.gaps:
            PrintColor(f"> for gap {gap}");
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap);
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}'];
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True);

        # cursor position shift
        PrintColor("\nEngineering cursor position data\n")
        for gap in self.gaps:
            PrintColor(f"> for gap {gap}");
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap);
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}'];
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}']);
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True);

        # word count shift
        PrintColor("\nEngineering word count data\n")
        for gap in self.gaps:
            PrintColor(f"> for gap {gap}");
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap);
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}'];
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}']);
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True);

        # get aggregate statistical features
        PrintColor("\nEngineering statistical summaries for features\n");

        feats_stat = \
        [
            ('event_id', ['max']),
            ('up_time', ['max']),
            ('action_time', ['max', 'sum', 'std']),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ('cursor_position', ['nunique', 'max', 'sum']),
            ('word_count', ['nunique', 'max', 'mean'])
        ];

        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'std', 'sum']),
                (f'cursor_position_change{gap}', ['max', 'std', 'sum']),
                (f'word_count_change{gap}', ['max','std', 'sum'])
            ])

        pbar = tqdm(feats_stat);

        for item in pbar:
            colname, methods = item[0], item[1];

            for method in methods:
                pbar.set_postfix();
                if isinstance(method, str):
                    method_name = method;
                else:
                    method_name = method.__name__;

                pbar.set_postfix(column=colname, method=method_name);
                tmp_df = \
                df.groupby(['id']).\
                agg({colname: method}).\
                reset_index().\
                rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left');

        # counts
        PrintColor("\nEngineering activity counts data\n");
        tmp_df = self.activity_counts(df);
        feats = pd.concat([feats, tmp_df], axis=1);

        PrintColor("\nEngineering event counts data\n");
        tmp_df = self.event_counts(df, 'down_event');
        feats  = pd.concat([feats, tmp_df], axis=1);
        tmp_df = self.event_counts(df, 'up_event');
        feats  = pd.concat([feats, tmp_df], axis=1);

        PrintColor("\nEngineering text change counts data\n");
        tmp_df = self.text_change_counts(df);
        feats  = pd.concat([feats, tmp_df], axis=1);

        PrintColor("\nEngineering punctuation counts data\n");
        tmp_df = self.match_punctuations(df);
        feats  = pd.concat([feats, tmp_df], axis=1);

        # input words
        PrintColor("\nEngineering input words data\n");
        tmp_df = self.get_input_words(df);
        feats = pd.merge(feats, tmp_df, on='id', how='left');

        # compare feats
        PrintColor("\nEngineering ratios data\n");
        feats['word_time_ratio']  = feats['word_count_max'] / feats['up_time_max'];
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max'];
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max'];
        feats['idle_time_ratio']  = feats['action_time_gap1_sum'] / feats['up_time_max'];

        PrintColor("\nDone!\n");
        return feats;

print();
collect();
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED);



RAM memory GB usage = 0.2847
CPU times: user 196 ms, sys: 813 µs, total: 197 ms
Wall time: 234 ms
